In [2]:
import pandas as pd

### Yelp Data

In [3]:
df = pd.read_csv("~/real_estate/data/v6/semisupervised_df.csv")

In [4]:
features = [
    "median_sqft_value","percent_decreasing","percent_increasing","zri_sqft_value","zhvi_condo",
    "zhvi_singlefam","mean_rating","bar_review_count","restaurant_review_count","coffee_review_count",
    "price_2_review_count","price_3_review_count","review_count","dependent_count_lag",
    "sum_adjusted_income_lag","sum_real_estate_taxes_lag","sum_mortgage_interest_paid_lag",
    "tax_return_count_lag","joint_return_count_lag","adult_count_lag","people_count_lag",
    "mean_adjusted_income_lag","mean_real_estate_taxes_hh_lag","mean_mortgage_interest_hh_lag",
    "mean_adjusted_income_with_dep_lag","90_day_treasury_bill_rate","mean_hourly_earnings_adj",
    "mean_fixed_mortgage_rate","total_nonfarm_payroll"
]

target = "zhvi_condo"

# Store data from previous, future months and years
feature_mo = {}
feature_yr = {}
zhvi_condo_12_months = []

for i in features:
    feature_mo[i+"_mo"] = []
    feature_yr[i+"_yr"] = []

In [5]:
for i in df.index:
    
    current_row = df.iloc[i]
    region_name = current_row["postal_code"]
    month = int(current_row["month"])
    year = int(current_row["year"])
    next_year = year + 1
    
    # Get column values of previous time preiods
    last_year = year - 1
    if month == 1:
        last_month_month = 12
        last_month_year = year - 1
    else:
        last_month_month = month - 1
        last_month_year = year
    
    last_month_row = df.loc[(df["postal_code"] == region_name) & 
                            (df["month"] == last_month_month) &
                            (df["year"] == last_month_year)]
    
    last_year_row = df.loc[(df["postal_code"] == region_name) &
                           (df["month"] == month) &
                           (df["year"] == last_year)]
    
    next_year_row = df.loc[(df["postal_code"] == region_name) &
                           (df["month"] == month) &
                           (df["year"] == next_year)]
    
    for i in features:
        
        feature_mo[i+"_mo"].append(last_month_row[i].values)
        feature_yr[i+"_yr"].append(last_year_row[i].values)
        
    next_year_target = next_year_row["zhvi_condo"]
    
    if len(next_year_target) == 0:
        next_year_target = None
    else:
        next_year_target = next_year_target.values[0]
    
    zhvi_condo_12_months.append(next_year_target)

In [6]:
# Add data to dataframe
for i in feature_mo.keys():
    new_list = [j[0] if len(j) > 0 else None for j in feature_mo[i]]
    feature_mo[i] = new_list

for i in feature_yr.keys():
    new_list = [j[0] if len(j) > 0 else None for j in feature_yr[i]]
    feature_yr[i] = new_list
    
for i in feature_mo.keys():
    df[i] = feature_mo[i]
    
for i in feature_yr.keys():
    df[i] = feature_yr[i]
    
df["zhvi_condo_12_months"] = zhvi_condo_12_months

In [8]:
# Filter
df = df.dropna()

# Export
df.to_csv("~/real_estate/data/v6/yelp_supervised_df.csv", index=False)

In [9]:
print(df.shape)
df.head()

(15892, 91)


,median_sqft_value,percent_decreasing,percent_increasing,zri_sqft_value,zhvi_condo,zhvi_singlefam,month,year,mean_rating,bar_review_count,...,sum_mortgage_interest_paid_lag_yr,mean_real_estate_taxes_hh_lag_yr,bar_review_count_yr,price_2_review_count_yr,mean_fixed_mortgage_rate_yr,review_count_yr,median_sqft_value_yr,percent_decreasing_yr,90_day_treasury_bill_rate_yr,zhvi_condo_12_months
13,555.0,49.10,31.85,2.946,509700.0,600453.621526,2.0,2011.0,5.00,0.0,...,15431.809,4.039578,0.0,0.0,4.9900,1.0,583.0,12.27,0.11,514400.0
14,556.0,44.28,39.23,2.944,511300.0,596095.098691,3.0,2011.0,5.00,0.0,...,15431.809,4.039578,0.0,0.0,4.9675,1.0,585.0,12.35,0.15,517000.0
15,560.0,40.59,41.49,2.940,513800.0,606783.350410,4.0,2011.0,5.00,0.0,...,15431.809,4.039578,0.0,0.0,5.0980,1.0,582.0,12.12,0.16,520500.0
16,564.0,42.62,39.98,2.960,515500.0,601942.108697,5.0,2011.0,4.75,0.0,...,15431.809,4.039578,0.0,0.0,4.8875,2.0,578.0,17.09,0.16,523600.0
17,564.0,44.95,36.07,3.000,517900.0,597623.524127,6.0,2011.0,4.75,0.0,...,15431.809,4.039578,0.0,0.0,4.8875,2.0,578.0,17.09,0.16,523600.0
